<a href="https://colab.research.google.com/github/crisbpadilla/CapstoneProject-1-M2M/blob/main/CapstoneProject_2_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np

concrete=pd.read_csv('concrete.csv')


# Preprocessing data

In [11]:
concrete_pluscat=concrete.copy()
# cuales son los escenarios? 9 (matriz 3x3) , la mitad menos 1 ~ 3
#a) slag-ash b)slag-superplastic c)ash-superplastic

concrete_pluscat['type_mix'] = 'all mix'
concrete_pluscat.loc[
    (concrete_pluscat['slag']>0) & (concrete_pluscat[['superplastic','ash']] == 0).all(axis=1),
    'type_mix',
] = 'slag'
concrete_pluscat.loc[
    (concrete_pluscat['ash']>0) & (concrete_pluscat[['superplastic','slag']] == 0).all(axis=1),
    'type_mix',
] = 'ash'
concrete_pluscat.loc[
    (concrete_pluscat['superplastic']>0) & (concrete_pluscat[['slag','ash']] == 0).all(axis=1),
    'type_mix',
] = 'superplastic'

concrete_pluscat.loc[
    (concrete_pluscat[['slag', 'ash']].gt(0).all(axis=1)) & (concrete_pluscat['superplastic'] == 0),
    'type_mix',
] = 'slag-ash'

concrete_pluscat.loc[
    (concrete_pluscat[['slag', 'superplastic']].gt(0).all(axis=1)) & (concrete_pluscat['ash'] == 0),
    'type_mix',
] = 'slag-superplastic'

concrete_pluscat.loc[
    (concrete_pluscat[['ash', 'superplastic']].gt(0).all(axis=1)) & (concrete_pluscat['slag'] == 0),
    'type_mix',
] = 'ash-superplastic'
concrete_pluscat.loc[
    (concrete_pluscat['cement']>0) & (concrete_pluscat[['superplastic','slag','ash']] == 0).all(axis=1),
    'type_mix',
] = 'just cement'

In [12]:
# Create the dataframe with all the numerical and categorical values
concrete_data=pd.get_dummies(data=concrete_pluscat,columns=['type_mix'],drop_first=True).astype(int)


In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X=concrete_data.drop(columns=['strength'])
y=concrete_data['strength']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)


In [14]:
# Seprating my numeric variables and nominal/categorical values
num_col=[a for a in concrete.columns]
num_col.remove('strength')
#numerical values
X_train_num=X_train[num_col]
X_test_num=X_test[num_col]
#nominal values
X_train_nom=X_train.drop(columns=num_col)
X_test_nom=X_test.drop(columns=num_col)

##Scalling the data

In [37]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scale_data=StandardScaler()
print(pd.DataFrame(scale_data.fit_transform(X_train_num)))

X_train_num_scale=scale_data.fit_transform(X_train_num)
X_test_num_scale=scale_data.fit_transform(X_test_num)


            0         1         2         3         4         5         6  \
0   -0.923421  0.562688 -0.847935  1.009923 -0.998089 -0.189745  0.352652   
1    1.592888 -0.566329  0.375830 -0.924602  0.867588 -0.088073 -0.762182   
2   -0.234545 -0.842823 -0.847935  0.490904 -0.998089 -1.079369  2.189595   
3   -1.095639  0.631812  1.196837 -0.311216  0.358767  0.394865 -0.331451   
4   -0.071894 -0.842823  0.530737  0.774006  0.697981 -0.545594 -0.128753   
..        ...       ...       ...       ...       ...       ...       ...   
767 -1.086072 -0.842823  1.677049 -0.264032 -0.319661  1.068438  0.099281   
768 -0.263248 -0.842823  0.670154 -1.679538  1.376409  0.166105  1.480155   
769  1.592888 -0.566329  0.375830 -0.924602  0.867588 -0.088073 -0.762182   
770  1.047528  1.334568 -0.847935 -1.726722  2.733265 -0.380379 -0.217433   
771  1.391966  0.378359 -0.847935 -1.349253  1.715623 -1.549599  1.454818   

            7  
0   -0.681777  
1   -0.615251  
2   -0.615251  
3    0.1996

In [55]:
#lineal regession gives a r2 too low and the function seems to fit to a log function
from sklearn.preprocessing import PolynomialFeatures,FunctionTransformer
from sklearn import linear_model
from sklearn.pipeline import make_pipeline

cuadratic = PolynomialFeatures(degree=3, include_bias=False)
X_train_pca_cuadratic = cuadratic.fit_transform(X_train_num_scale)
X_test_pca_cuadratic = cuadratic.fit_transform(X_test_num_scale)

reg=linear_model.LinearRegression()
reg.fit(X_train_pca_cuadratic,y_train)
prediction_train=reg.predict(X_train_pca_cuadratic)
print(reg.score(X_train_pca_cuadratic,y_train))
print(reg.score(X_test_pca_cuadratic,y_test))


0.9360852948021252
0.75441258214008


In [62]:
from bokeh.plotting import figure, output_notebook, show, output_notebook
from bokeh.models import ColumnDataSource

output_notebook()

csource=ColumnDataSource(data={'x':X_train_pca,'y':y_train,'y_t':prediction_train})
c=figure(title='test graph')
c.scatter(x='x',y='y',source=csource)
c.scatter(x='x',y='y_t',source=csource,color='red')
show(c)

In [57]:


X_train_ln_full = np.hstack([X_train_pca_cuadratic, X_train_nom])
X_test_ln_full = np.hstack([X_test_pca_cuadratic, X_test_nom])

X_train_ln_full.shape

(772, 170)

In [58]:
ln_model= linear_model.RidgeCV()
ln_model.fit(X_train_ln_full,y_train)
print(ln_model.score(X_train_ln_full,y_train))
print(ln_model.score(X_test_ln_full,y_test))

0.93215885303758
0.8068956501918065


It is clear that the compressive strength is a value given by multiple variables, one of them are base of the mix and others are added to the mix , which allowed to improved properties of the solution so it improves the development time of the strength and its strength too.

 so, it is a multivariable model.